In [55]:
from gensim.models import Word2Vec
model = Word2Vec.load("300features_40minwords_10context")

In [69]:
import pandas as pd

# Read data from files
train = pd.read_csv("labeledTrainData.tsv", header=0, delimiter="\t", quoting = 3)
test = pd.read_csv( "testData.tsv", header=0, delimiter="\t", quoting=3 )
unlabeled_train = pd.read_csv("unlabeledTrainData.tsv", header=0, delimiter="\t", quoting = 3)

In [ ]:
# Set values for various parameters

num_features = 300 # Word vector dimensionality
min_word_count = 40 # Minimum word count
num_workers = 4 # Threads
context = 10 # Context window size

downsampling = 1e-3 # Downsampling setting fro frequent words

In [71]:
# Import various modules for string cleaning
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def review_to_wordlist( review, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review,"lxml").get_text()
    #  
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)

In [72]:
# The syn0 in model is moved to model.wv.vectors
type(model.wv.vectors)

numpy.ndarray

In [73]:
print(model.wv.vectors.shape)

(16490, 300)


In [74]:
print(model.wv["flower"])

[-5.06587438e-02 -1.30716218e-02 -2.69954707e-02  6.33645728e-02
 -6.51245043e-02 -6.32605106e-02 -4.81012336e-04 -1.11888098e-02
 -7.76432529e-02 -4.86407056e-02  9.42130908e-02  5.38372397e-02
  5.48828393e-02  7.37902569e-03 -1.48566850e-02  2.60501076e-02
 -5.07104732e-02  1.32804230e-01  2.36674372e-04 -6.28971010e-02
  5.41811902e-03  1.13381026e-02 -9.77564007e-02 -2.56607942e-02
 -4.20020781e-02  6.11054152e-02  7.67687634e-02  2.49413797e-03
  2.04210775e-03 -1.03719108e-01  1.49613708e-01  4.36168276e-02
  4.21950445e-02  6.13381118e-02 -3.76603231e-02 -2.22360268e-02
 -6.94532022e-02 -1.18287876e-01  3.16194966e-02  6.24483339e-02
 -7.27119520e-02  4.38906811e-02  3.55872139e-02 -7.94034302e-02
  5.18171936e-02 -4.87119611e-03  6.46155477e-02  5.70900179e-02
 -9.54613760e-02 -3.84350047e-02  6.64945915e-02  6.05421513e-03
 -2.43431292e-02  9.96646360e-02 -2.78943069e-02  1.27462600e-03
 -7.82098528e-03  1.18875699e-02  2.63399892e-02  3.64927538e-02
  1.58986285e-01  4.92288

In [75]:
import numpy as np

In [81]:
def makeFeatureVec(words, model, num_features):

    # Pre-initialize an empty numpy array for speed
    featureVec = np.zeros((num_features,),dtype="float32")
    nwords = 0
    
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.wv.index2word)
    
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set:
            nwords = nwords+1.
            featureVec = np.add(featureVec, model[word])
    
    # Divide the result by the number of words to get the avg
    featureVec = np.divide(featureVec, nwords)
    return featureVec
    

In [82]:
def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0
    counter = int(counter)
    
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    
    # Loop through the reviews
    for review in reviews:
       #
       # Print a status message every 1000th review
        if counter%1000. == 0.:
            print("Review %d of %d" % (counter, len(reviews)))
        
       # Call the function (defined above) that makes average feature vectors, increment the counter
        reviewFeatureVecs[counter] = makeFeatureVec(review, model, num_features)
        counter = counter + 1
        
    return reviewFeatureVecs

In [83]:
# ****************************************************************
# Calculate average feature vectors for training and testing sets,
# using the functions we defined above. Notice that we now use stop word
# removal.

print ("Creating average feature vecs for Training reviews")
clean_train_reviews = []
for review in train["review"]:
    clean_train_reviews.append( review_to_wordlist( review, \
        remove_stopwords=True ))

trainDataVecs = getAvgFeatureVecs( clean_train_reviews, model, num_features )

Creating average feature vecs for Training reviews
Review 0 of 25000


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  app.launch_new_instance()


Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in true_divide


Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 19000 of 25000
Review 20000 of 25000
Review 21000 of 25000
Review 22000 of 25000
Review 23000 of 25000
Review 24000 of 25000


In [84]:
print ("Creating average feature vecs for test reviews")
clean_test_reviews = []
for review in test["review"]:
    clean_test_reviews.append( review_to_wordlist( review, \
        remove_stopwords=True ))

testDataVecs = getAvgFeatureVecs( clean_test_reviews, model, num_features )

Creating average feature vecs for test reviews
Review 0 of 25000


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  app.launch_new_instance()
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in true_divide


Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 19000 of 25000
Review 20000 of 25000
Review 21000 of 25000
Review 22000 of 25000
Review 23000 of 25000
Review 24000 of 25000


In [95]:
# Fit a random forest to the training data, using 100 trees
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import Imputer

forest = RandomForestClassifier( n_estimators = 100 )

print("Fitting a random forest to labeled training data...")
trainDataVecs = Imputer().fit_transform(trainDataVecs)

forest = forest.fit( trainDataVecs, train["sentiment"] )

Fitting a random forest to labeled training data...


In [97]:
# Test & extract results 
testDataVecs = Imputer().fit_transform(testDataVecs)
result = forest.predict( testDataVecs )

# Write the test results 
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )
output.to_csv( "Word2Vec_AverageVectors.csv", index=False, quoting=3 )

In [98]:
from sklearn.cluster import KMeans
import time
start = time.time() # Start time

In [103]:
# Set "k" (num_clusters) to be 1/5th of the vocabulary size, or an
# average of 5 words per cluster
word_vectors = model.wv.vectors
num_clusters = int(word_vectors.shape[0] / 5)

In [104]:
# Initalize a k-means object and use it to extract centroids
kmeans_clustering = KMeans( n_clusters = num_clusters )
idx = kmeans_clustering.fit_predict( word_vectors )

In [106]:
# Get the end time and print how long the process took
end = time.time()
elapsed = end - start
print("Time taken for K Means clustering: ", elapsed, "seconds.")

Time taken for K Means clustering:  1180.7468872070312 seconds.


In [108]:
# Create a Word / Index dictionary, mapping each vocabulary word to
# a cluster number 
word_centroid_map = dict(zip( model.wv.index2word, idx ))

In [116]:
# For the first 10 clusters
for cluster in range(0,10):
    #
    # Print the cluster number  
    print("\nCluster %d" % cluster)
    #
    # Find all of the words for that cluster number, and print them out
    words = []
    for i in range(0,len(list(word_centroid_map.values()))):
        if( list(word_centroid_map.values())[i] == cluster ):
            words.append(list(word_centroid_map)[i])
    print(words)


Cluster 0
['bent']

Cluster 1
['gleefully', 'glee', 'vomiting', 'leering', 'teasing', 'orville']

Cluster 2
['olivia', 'havilland']

Cluster 3
['gunslinger', 'scrimm', 'lawman', 'wily']

Cluster 4
['diversity']

Cluster 5
['backs', 'throats', 'butts', 'noses', 'beds', 'pockets', 'ropes', 'tongues', 'necks']

Cluster 6
['bond', 'stewart', 'cagney', 'cameron', 'mason', 'garner', 'belushi', 'coburn', 'gleason', 'caan', 'russo', 'gandolfini', 'brolin', 'toback', 'spader', 'caviezel', 'duval', 'howe']

Cluster 7
['romania', 'denmark', 'netherlands', 'serbia']

Cluster 8
['maintains', 'maintained', 'entertains', 'retains', 'balances']

Cluster 9
['hopkins', 'dern', 'perkins', 'viggo', 'mortensen', 'elisha', 'christensen']


In [117]:
def create_bag_of_centroids( wordlist, word_centroid_map ):
    #
    # The number of clusters is equal to the highest cluster index
    # in the word / centroid map
    num_centroids = max( word_centroid_map.values() ) + 1
    #
    # Pre-allocate the bag of centroids vector (for speed)
    bag_of_centroids = np.zeros( num_centroids, dtype="float32" )
    #
    # Loop over the words in the review. If the word is in the vocabulary,
    # find which cluster it belongs to, and increment that cluster count 
    # by one
    for word in wordlist:
        if word in word_centroid_map:
            index = word_centroid_map[word]
            bag_of_centroids[index] += 1
    #
    # Return the "bag of centroids"
    return bag_of_centroids

In [118]:
# Pre-allocate an array for the training set bags of centroids (for speed)
train_centroids = np.zeros( (train["review"].size, num_clusters), \
    dtype="float32" )

# Transform the training set reviews into bags of centroids
counter = 0
for review in clean_train_reviews:
    train_centroids[counter] = create_bag_of_centroids( review, \
        word_centroid_map )
    counter += 1

In [119]:
# Repeat for test reviews 
test_centroids = np.zeros(( test["review"].size, num_clusters), \
    dtype="float32" )

counter = 0
for review in clean_test_reviews:
    test_centroids[counter] = create_bag_of_centroids( review, \
        word_centroid_map )
    counter += 1

In [121]:
# Fit a random forest and extract predictions 
forest = RandomForestClassifier(n_estimators = 100)

# Fitting the forest may take a few minutes
print("Fitting a random forest to labeled training data...")
forest = forest.fit(train_centroids,train["sentiment"])
result = forest.predict(test_centroids)

# Write the test results 
output = pd.DataFrame(data={"id":test["id"], "sentiment":result})
output.to_csv( "BagOfCentroids.csv", index=False, quoting=3 )

Fitting a random forest to labeled training data...
